In [ ]:
#Median Absolute Deviation Outlier Detection Function 

#this function uses MAD and a user defined threshold to detect outliers, it also deseasonalizes the series using STL decomposition. 


def rolling_med_outliers(ts, # feed in timeseries(s)
                         target, #feed in column to predict outliers on 
                         window_size=13, # present for monthly periodicity as months + 1 for smoothing 
                         alpha=3.5, # adjust this for sensitivity of outlier detection 
                         show_plot=False):
       #imports 
    from statsmodels.robust import mad
    plt.style.use('fivethirtyeight')
    
#fit STL decomposition to deseasonalize the series, only catching outliers that lie outside the seasonal cycle. 
    res = STL(data[target],
                robust=True).fit()

    deseason = (data[target] - res.seasonal).to_frame(target) 

    new_data = deseason.join(deseason[target]\
    .rolling(window_size,
                
                center=True)\
    .agg({'rolling_median':'median',
            'rolling_mad':mad}))

    #create new dataframe (upper and lower bounds + outlier flag)

    new_data = new_data.assign(lower = new_data['rolling_median'] - new_data['rolling_mad'] * alpha,
                            upper = new_data['rolling_median'] + new_data['rolling_mad'] * alpha,
                            is_outlier = np.abs(new_data[target] - new_data['rolling_median']) > alpha * new_data['rolling_mad'])
        
    
    if show_plot == True: 

        ax = new_data[[target,'rolling_median']].plot(figsize=(15,8),
                            marker='.',
                            alpha=.5)

        new_data[['upper','lower']].plot(ax=ax,
                                    alpha=.4,
                                    color='black')


        #plot outliers 

        if new_data['is_outlier'].any():
            
            new_data[target].loc[new_data['is_outlier']].plot(marker='o',
                                                        color='r',
                                                        ax=ax,
                                                        alpha=.5,
                                                        linestyle='')

        ax.set_title(f'Outlier Detection (Robust) (alpha = {alpha})')
    
    return new_data